# Abandoned

In [1]:
from qib.operator import FieldOperator, FieldOperatorTerm, IFOType, IFODesc
import fermitensor as ftn # after installing fermitensor (installed with kernel 3.10.11)

import hamiltonian_eq_9 as ham
import matrix_reference as ref

import numpy as np
# from scipy import sparse
from scipy.sparse.linalg import norm
from typing import Sequence

In [2]:
L = 8
L_1 = L//2
A = range(L_1)
B = range(L_1, L)

# latt = qib.lattice.FullyConnectedLattice((L,))
# field = qib.field.Field(qib.field.ParticleType.FERMION, latt)
# tkin, vint = construct_random_coefficients(L)

# create MolecularHamiltonian Object
H_ref = ref.construct_random_molecular_hamiltonian(L)
field = H_ref.field
tkin = H_ref.tkin
vint = H_ref.vint


### Test P

In [12]:
def get_P_ref(field, vint, L, a: IFOType, A, b: IFOType, B):
    V = FieldOperatorTerm([IFODesc(field, a),
                            IFODesc(field, b),
                            IFODesc(field, IFOType.FERMI_ANNIHIL),
                            IFODesc(field, IFOType.FERMI_ANNIHIL)],
                            get_range(vint, [A, B, L, L]).transpose((0, 1, 3, 2)))
    return FieldOperator([V])

### Test Q

In [13]:
def get_Q_ref(field, vint, L, a: IFOType, A, b: IFOType, B):
    V = FieldOperatorTerm([IFODesc(field, a),
                            IFODesc(field, b),
                            IFODesc(field, IFOType.FERMI_CREATE),
                            IFODesc(field, IFOType.FERMI_ANNIHIL)],
                            get_range(vint, [A, L, B, L]).transpose((0, 2, 1, 3))
                              - get_range(vint, [A, L, L, B]).transpose((0, 2, 3, 1)))
    return FieldOperator([V])

In [ ]:
Q_ref = get_Q_ref(field, vint, B, IFOType.FERMI_CREATE, A, IFOType.FERMI_ANNIHIL, A)
Q_v = np.zeros(shape=(L,L,L,L), dtype=np.complex128)
a = single_term_FO(field, [IFOType.FERMI_CREATE, IFOType.FERMI_ANNIHIL], [A, A])
print("a shape: ", a.terms[0].coeffs.shape)

for i in range(A[1]):
    for j in range(A[1]):
        print(i)
        
        Q = get_Q_as_field_operator(field, vint, B, i, j)
        Q_v[i][j] = Q.terms[0].coeffs
        
        # print(Q.terms[0].coeffs)
        # print(Q_ref.terms[0].coeffs[i])
        
        print("Q shape: ", Q.terms[0].coeffs.shape)
        print("Q opdesc: ", len(Q.terms[0].opdesc))
        print("Q-ref shape: ", Q_ref.terms[0].coeffs.shape)
        print("Q-ref opdesc: ", len(Q_ref.terms[0].opdesc))

        # S = multiply_element_wise(a, S_i)
        # print(S.shape)

        # print()

print(Q_v - Q_ref.terms[0].coeffs)
Q = FieldOperator([FieldOperatorTerm(a.terms[0].opdesc + Q.terms[0].opdesc, Q_v)])
print(Q.terms[0].opdesc) #  == S_ref.terms[0].opdesc
print(Q_ref.terms[0].opdesc)
assert norm(Q.as_matrix()-Q_ref.as_matrix()) == 0

In [ ]:
# S = np.zeros_like((L,L))
for i in range(A[1]):
    print(i)
    a = single_term_FO(field, [IFOType.FERMI_CREATE, IFOType.FERMI_ANNIHIL], [A,A])
    print("a shape: ", a.terms[0].coeffs.shape)
    # print("b shape: ", b.terms[0].coeffs.shape)
    
    

### Test S

In [ ]:
def get_S_ref(field, tkin, vint, L, a: IFOType, A):
    # kinetic hopping term
    T = FieldOperatorTerm([IFODesc(field, a),
                            IFODesc(field, IFOType.FERMI_ANNIHIL)],
                            get_range(tkin, [A, L]))
    # interaction term
    V = FieldOperatorTerm([IFODesc(field, a),
                            IFODesc(field, IFOType.FERMI_CREATE),
                               IFODesc(field, IFOType.FERMI_ANNIHIL),
                            IFODesc(field, IFOType.FERMI_ANNIHIL)],
                            get_range(vint, [A, L, L, L]).transpose((0, 1, 3, 2)))
    return FieldOperator([T, V])

In [ ]:
S_t = np.zeros(shape=(L,L), dtype=np.complex128)
S_v = np.zeros(shape=(L,L,L,L), dtype=np.complex128)

for i in range(A[1]):
    S_i = get_S_as_field_operator(field, tkin, vint, B, i)
    S_t[i] = S_i.terms[0].coeffs
    S_v[i] = S_i.terms[1].coeffs

S = FieldOperator([FieldOperatorTerm([IFOType.FERMI_CREATE] + S_i.terms[0].opdesc, S_t), 
                   FieldOperatorTerm([IFOType.FERMI_CREATE] + S_i.terms[1].opdesc, S_v)])

In [ ]:
S_ref = get_S_ref(field, tkin, vint, B, IFOType.FERMI_CREATE, A)
S_t = np.zeros(shape=(L,L), dtype=np.complex128)
S_v = np.zeros(shape=(L,L,L,L), dtype=np.complex128)
a = single_term_FO(field, [IFOType.FERMI_CREATE], [A])
print("a shape: ", a.terms[0].coeffs.shape)

for i in range(A[1]):
    # print(i)
    
    S_i = get_S_as_field_operator(field, tkin, vint, B, i)
    S_t[i] = S_i.terms[0].coeffs
    S_v[i] = S_i.terms[1].coeffs
    
    # print(S_i.terms[0].coeffs)
    # print(type(S_ref.terms[0].coeffs[i][0]))

    # print("S shape: ", S_i.terms[0].coeffs.shape)
    # print("S opdesc: ", len(S_i.terms[0].opdesc))
    # print("S-ref shape: ", S_ref.terms[0].coeffs.shape)
    # print("S-ref opdesc: ", len(S_ref.terms[0].opdesc))

    # S = multiply_element_wise(a, S_i)
    # print(S.shape)

    # print()

print(S_t - S_ref.terms[0].coeffs)
print(S_v - S_ref.terms[1].coeffs)
S = FieldOperator([FieldOperatorTerm(a.terms[0].opdesc + S_i.terms[0].opdesc, S_t), 
                   FieldOperatorTerm(a.terms[0].opdesc + S_i.terms[1].opdesc, S_v)])
print(S.terms[0].opdesc) #  == S_ref.terms[0].opdesc
print(S_ref.terms[0].opdesc)
assert norm(S.as_matrix()-S_ref.as_matrix()) == 0

### Abandoned Stuff

In [ ]:
def get_P_as_field_operator(H, L, a: IFOType, A, b: IFOType, B):
    V = FieldOperatorTerm([IFODesc(H.field, a),
                            IFODesc(H.field, b),
                            IFODesc(H.field, IFOType.FERMI_ANNIHIL),
                            IFODesc(H.field, IFOType.FERMI_ANNIHIL)],
                            H.vint[A[0]:A[1], B[0]:B[1], L[0]:L[1], L[0]:L[1]].transpose((0, 1, 3, 2)))
    return FieldOperator([V])
    
def get_Q_as_field_operator(H, L, a: IFOType, A, b: IFOType, B):
    V = FieldOperatorTerm([IFODesc(H.field, a),
                            IFODesc(H.field, b),
                            IFODesc(H.field, IFOType.FERMI_CREATE),
                            IFODesc(H.field, IFOType.FERMI_ANNIHIL)],
                            H.vint[A[0]:A[1], L[0]:L[1], B[0]:B[1], L[0]:L[1]].transpose((0, 2, 1, 3))
                              - H.vint[A[0]:A[1], L[0]:L[1], L[0]:L[1], B[0]:B[1]].transpose((0, 2, 3, 1)))
    return FieldOperator([V])

def get_S_as_field_operator(H, L, a: IFOType, A):
    # kinetic hopping term
    T = FieldOperatorTerm([IFODesc(H.field, a),
                            IFODesc(H.field, IFOType.FERMI_ANNIHIL)],
                            H.tkin[A[0]:A[1], L[0]:L[1]])
    # interaction term
    V = FieldOperatorTerm([IFODesc(H.field, a),
                            IFODesc(H.field, IFOType.FERMI_CREATE),
                            IFODesc(H.field, IFOType.FERMI_ANNIHIL),
                            IFODesc(H.field, IFOType.FERMI_ANNIHIL)],
                            H.vint[A[0]:A[1], L[0]:L[1], L[0]:L[1], L[0]:L[1]].transpose((0, 1, 3, 2)))
    return FieldOperator([T, V])

In [ ]:
def prepend_IFO(P: FieldOperator, a: IFOType, L):
    # prepend IFO to a Field Operator
    # TODO how does that work with the dimensions of coeff???
    for term in P.terms:
        term.opdesc = (IFODesc(term.opdesc[0].field, a),) + term.opdesc # TODO Case possible with empty list?

In [ ]:
# a_i^At S_i^B i in A
S = get_S_as_field_operator(H, B, IFOType.FERMI_CREATE, A)
for term in S.terms:
    print(term)
    print(term.opdesc)
print(S.is_hermitian())

In [ ]:
def single_term_FO_old(field, types: Sequence[IFOType]):
    opdesc = []
    for type in types:
        opdesc.append(IFODesc(field, type))
    return FieldOperator([FieldOperatorTerm(opdesc, [1]*len(opdesc))]) # TODO dimensions correct?

In [ ]:
#print("tkin: ", get_range(tkin, [A,A]))
print("tkin: ", get_range(tkin, [A,A]).shape)
a = single_term_FO(field, [IFOType.FERMI_CREATE], [A])
print("a: ", a.terms[0].coeffs.shape)
b = single_term_FO(field, [IFOType.FERMI_CREATE, IFOType.FERMI_CREATE], [A, B])
print("b: ", b.terms[0].coeffs.shape)

In [ ]:
Test = np.random.rand(2,2,2)
# print(Test[1,:,])
print(Test[1,:,].shape)
print("range: ", get_range(Test, [0,1]))